In [ ]:
#normal stuff
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
from scipy import stats
from sklearn.preprocessing import StandardScaler
sns.set()
%matplotlib inline

#special effects
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Lars, ElasticNet
from sklearn.svm import SVR
from sklearn.ensemble import AdaBoostRegressor
from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer, TransformedTargetRegressor
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder, RobustScaler
from sklearn.decomposition import PCA
from sklearn.metrics import mean_absolute_error

In [ ]:
#Dataloader
path =[r"..\data\train_preprocessed.csv", r"..\data\test_preprocessed.csv"]

X = pd.read_csv(path[0])
X_test = pd.read_csv(path[1])

y=X["logSalePrice"]
X.drop("logSalePrice", axis=1, inplace=True)

print(X.shape, y.shape, X_test.shape)


In [ ]:


X_train, X_valid, y_train, y_valid = train_test_split( X, y, train_size = 0.8, test_size=0.2, random_state=0)


XGBR_final = XGBRegressor(n_estimators=4500, learning_rate=0.008, max_depth = 4, min_child_weight=4, gamma=0, subsample= 0.940136255964662, colsample_bytree = 0.5, alpha = 0.10925528556796542, early_stopping_counts=50, evals=y_valid, eval_metric="mae",cv=True, seed=0, verbosity=1 )


my_pipe_out = Pipeline([#("preprocessing", preprocessing),
                        ("rb_scaler", RobustScaler()),
                        ("reg_final", XGBR_final)
                        ])


my_pipe_out.fit(X_train, y_train)



valid_preds = my_pipe_out.predict(X_valid)
print(valid_preds)
valid_score = mean_absolute_error(np.exp(y_valid), np.exp(valid_preds))
print(valid_score)


In [ ]:
test_preds = my_pipe_out.predict(X_test)

#recalc preds
test_preds=np.exp(test_preds)
output = pd.DataFrame({"Id": X_test.index,
                     "SalePrice": test_preds})
output.to_csv("submission_other.csv", index=False)